In [1]:
## 导入第三方包
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_auc_score,mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
## 训练数据及测试数据准备
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train

,article_id,date,baike_id_1h,price,price_diff,author,level1,level2,level3,level4,...,buzhi_1h,favorite_1h,orders_1h,baike_id_2h,comments_2h,zhi_2h,buzhi_2h,favorite_2h,orders_2h,orders_3h_15h
0,356,1,104,5969.0,-597.8,21562,8,53,63,212,...,0,0,0,104,0,0,0,0,0,0
1,388,1,219783,190.0,NaN,51522,14,15,580,0,...,3,12,5,219783,11,3,1,8,5,5
2,872,1,92240,139.0,20.0,69683,7,22,0,0,...,0,0,0,92240,0,0,0,0,0,0
3,938,1,88894,99.0,NaN,63556,19,2,753,0,...,0,0,0,88894,0,0,0,0,0,0
4,1957,1,85380,779.0,10.0,63860,5,40,429,0,...,0,0,0,85380,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807652,1804751,116,219783,99.0,0.0,44655,0,6,713,0,...,0,0,0,219783,0,0,0,0,0,0
1807653,1805971,116,219783,217.0,NaN,68203,0,6,707,0,...,0,0,0,219783,0,0,0,0,0,0
1807654,1807148,116,219783,138.0,NaN,20391,19,4,763,0,...,0,0,0,219783,0,0,0,0,0,0
1807655,1807424,116,219783,559.0,9.0,22941,7,50,237,0,...,0,0,0,219783,0,0,0,0,0,0


In [4]:
test

,article_id,date,baike_id_1h,price,price_diff,author,level1,level2,level3,level4,...,zhi_1h,buzhi_1h,favorite_1h,orders_1h,baike_id_2h,comments_2h,zhi_2h,buzhi_2h,favorite_2h,orders_2h
0,1809360,117,219783,9.40,NaN,39424,19,31,744,0,...,0,0,0,0,219783,0,0,0,0,0
1,1809857,117,219783,149.60,-38.77,51888,19,30,750,0,...,0,0,0,0,219783,0,0,0,0,0
2,1810275,117,219783,97.00,41.00,8649,13,13,664,0,...,0,0,0,0,219783,0,0,0,0,0
3,1810622,117,68785,5.42,2.50,52372,3,5,27,798,...,0,0,1,1,68785,0,0,0,1,1
4,1811139,117,219783,79.00,NaN,20410,13,96,146,0,...,0,0,0,0,219783,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149584,1955932,123,219783,123.50,18.05,69329,8,57,84,0,...,0,0,1,0,219783,0,0,0,0,0
149585,1956580,123,29553,2.40,1.45,71272,19,2,754,1199,...,0,0,0,0,29553,0,0,0,0,0
149586,1956675,123,219783,310.80,NaN,665,13,96,1,0,...,0,0,0,0,219783,0,0,0,0,0
149587,1956922,123,49355,99.00,10.00,55420,5,43,319,0,...,0,0,2,0,49355,0,0,0,0,0


In [5]:
#频数特征

temp=train['author'].value_counts().to_dict()
train['author_cnt']=train['author'].map(temp)

temp=train['brand'].value_counts().to_dict()
train['brand_cnt']=train['brand'].map(temp)

temp=train['mall'].value_counts().to_dict()
train['mall_cnt']=train['mall'].map(temp)




temp=test['author'].value_counts().to_dict()
test['author_cnt']=test['author'].map(temp)

temp=test['brand'].value_counts().to_dict()
test['brand_cnt']=test['brand'].map(temp)

temp=test['mall'].value_counts().to_dict()
test['mall_cnt']=test['mall'].map(temp)

In [6]:
#聚合特征
#level1,level2,level3,level4
###############################################train

#计数
train['gp_author_level1_cnt']=train.groupby('author')['level1'].transform(lambda x: x.count())
train['gp_author_level2_cnt']=train.groupby('author')['level2'].transform(lambda x: x.count())
train['gp_author_level3_cnt']=train.groupby('author')['level3'].transform(lambda x: x.count())
train['gp_author_level4_cnt']=train.groupby('author')['level4'].transform(lambda x: x.count())

train['gp_mall_level1_cnt']=train.groupby('mall')['level1'].transform(lambda x: x.count())
train['gp_mall_level2_cnt']=train.groupby('mall')['level2'].transform(lambda x: x.count())
train['gp_mall_level3_cnt']=train.groupby('mall')['level3'].transform(lambda x: x.count())
train['gp_mall_level4_cnt']=train.groupby('mall')['level4'].transform(lambda x: x.count())

train['gp_brand_level1_cnt']=train.groupby('brand')['level1'].transform(lambda x: x.count())
train['gp_brand_level2_cnt']=train.groupby('brand')['level2'].transform(lambda x: x.count())
train['gp_brand_level3_cnt']=train.groupby('brand')['level3'].transform(lambda x: x.count())
train['gp_brand_level4_cnt']=train.groupby('brand')['level4'].transform(lambda x: x.count())

#平均
train['gp_author_level1_mean']=train.groupby('author')['level1'].transform(lambda x: x.mean())
train['gp_author_level2_mean']=train.groupby('author')['level2'].transform(lambda x: x.mean())
train['gp_author_level3_mean']=train.groupby('author')['level3'].transform(lambda x: x.mean())
train['gp_author_level4_mean']=train.groupby('author')['level4'].transform(lambda x: x.mean())

train['gp_mall_level1_mean']=train.groupby('mall')['level1'].transform(lambda x: x.mean())
train['gp_mall_level2_mean']=train.groupby('mall')['level2'].transform(lambda x: x.mean())
train['gp_mall_level3_mean']=train.groupby('mall')['level3'].transform(lambda x: x.mean())
train['gp_mall_level4_mean']=train.groupby('mall')['level4'].transform(lambda x: x.mean())

train['gp_brand_level1_mean']=train.groupby('brand')['level1'].transform(lambda x: x.mean())
train['gp_brand_level2_mean']=train.groupby('brand')['level2'].transform(lambda x: x.mean())
train['gp_brand_level3_mean']=train.groupby('brand')['level3'].transform(lambda x: x.mean())
train['gp_brand_level4_mean']=train.groupby('brand')['level4'].transform(lambda x: x.mean())

#中位数
train['gp_author_level1_median']=train.groupby('author')['level1'].transform(lambda x: x.median())
train['gp_author_level2_median']=train.groupby('author')['level2'].transform(lambda x: x.median())
train['gp_author_level3_median']=train.groupby('author')['level3'].transform(lambda x: x.median())
train['gp_author_level4_median']=train.groupby('author')['level4'].transform(lambda x: x.median())

train['gp_mall_level1_median']=train.groupby('mall')['level1'].transform(lambda x: x.median())
train['gp_mall_level2_median']=train.groupby('mall')['level2'].transform(lambda x: x.median())
train['gp_mall_level3_median']=train.groupby('mall')['level3'].transform(lambda x: x.median())
train['gp_mall_level4_median']=train.groupby('mall')['level4'].transform(lambda x: x.median())

train['gp_brand_level1_median']=train.groupby('brand')['level1'].transform(lambda x: x.median())
train['gp_brand_level2_median']=train.groupby('brand')['level2'].transform(lambda x: x.median())
train['gp_brand_level3_median']=train.groupby('brand')['level3'].transform(lambda x: x.median())
train['gp_brand_level4_median']=train.groupby('brand')['level4'].transform(lambda x: x.median())

#方差
train['gp_author_level1_var']=train.groupby('author')['level1'].transform(lambda x: x.var())
train['gp_author_level2_var']=train.groupby('author')['level2'].transform(lambda x: x.var())
train['gp_author_level3_var']=train.groupby('author')['level3'].transform(lambda x: x.var())
train['gp_author_level4_var']=train.groupby('author')['level4'].transform(lambda x: x.var())

train['gp_mall_level1_var']=train.groupby('mall')['level1'].transform(lambda x: x.var())
train['gp_mall_level2_var']=train.groupby('mall')['level2'].transform(lambda x: x.var())
train['gp_mall_level3_var']=train.groupby('mall')['level3'].transform(lambda x: x.var())
train['gp_mall_level4_var']=train.groupby('mall')['level4'].transform(lambda x: x.var())

train['gp_brand_level1_var']=train.groupby('brand')['level1'].transform(lambda x: x.var())
train['gp_brand_level2_var']=train.groupby('brand')['level2'].transform(lambda x: x.var())
train['gp_brand_level3_var']=train.groupby('brand')['level3'].transform(lambda x: x.var())
train['gp_brand_level4_var']=train.groupby('brand')['level4'].transform(lambda x: x.var())


#标准差
train['gp_author_level1_std']=train.groupby('author')['level1'].transform(lambda x: x.std())
train['gp_author_level2_std']=train.groupby('author')['level2'].transform(lambda x: x.std())
train['gp_author_level3_std']=train.groupby('author')['level3'].transform(lambda x: x.std())
train['gp_author_level4_std']=train.groupby('author')['level4'].transform(lambda x: x.std())

train['gp_mall_level1_std']=train.groupby('mall')['level1'].transform(lambda x: x.std())
train['gp_mall_level2_std']=train.groupby('mall')['level2'].transform(lambda x: x.std())
train['gp_mall_level3_std']=train.groupby('mall')['level3'].transform(lambda x: x.std())
train['gp_mall_level4_std']=train.groupby('mall')['level4'].transform(lambda x: x.std())

train['gp_brand_level1_std']=train.groupby('brand')['level1'].transform(lambda x: x.std())
train['gp_brand_level2_std']=train.groupby('brand')['level2'].transform(lambda x: x.std())
train['gp_brand_level3_std']=train.groupby('brand')['level3'].transform(lambda x: x.std())
train['gp_brand_level4_std']=train.groupby('brand')['level4'].transform(lambda x: x.std())

#最大值
train['gp_author_level1_max']=train.groupby('author')['level1'].transform(lambda x: x.max())
train['gp_author_level2_max']=train.groupby('author')['level2'].transform(lambda x: x.max())
train['gp_author_level3_max']=train.groupby('author')['level3'].transform(lambda x: x.max())
train['gp_author_level4_max']=train.groupby('author')['level4'].transform(lambda x: x.max())

train['gp_mall_level1_max']=train.groupby('mall')['level1'].transform(lambda x: x.max())
train['gp_mall_level2_max']=train.groupby('mall')['level2'].transform(lambda x: x.max())
train['gp_mall_level3_max']=train.groupby('mall')['level3'].transform(lambda x: x.max())
train['gp_mall_level4_max']=train.groupby('mall')['level4'].transform(lambda x: x.max())

train['gp_brand_level1_max']=train.groupby('brand')['level1'].transform(lambda x: x.max())
train['gp_brand_level2_max']=train.groupby('brand')['level2'].transform(lambda x: x.max())
train['gp_brand_level3_max']=train.groupby('brand')['level3'].transform(lambda x: x.max())
train['gp_brand_level4_max']=train.groupby('brand')['level4'].transform(lambda x: x.max())

#最小值
train['gp_author_level1_min']=train.groupby('author')['level1'].transform(lambda x: x.min())
train['gp_author_level2_min']=train.groupby('author')['level2'].transform(lambda x: x.min())
train['gp_author_level3_min']=train.groupby('author')['level3'].transform(lambda x: x.min())
train['gp_author_level4_min']=train.groupby('author')['level4'].transform(lambda x: x.min())

train['gp_mall_level1_min']=train.groupby('mall')['level1'].transform(lambda x: x.min())
train['gp_mall_level2_min']=train.groupby('mall')['level2'].transform(lambda x: x.min())
train['gp_mall_level3_min']=train.groupby('mall')['level3'].transform(lambda x: x.min())
train['gp_mall_level4_min']=train.groupby('mall')['level4'].transform(lambda x: x.min())

train['gp_brand_level1_min']=train.groupby('brand')['level1'].transform(lambda x: x.min())
train['gp_brand_level2_min']=train.groupby('brand')['level2'].transform(lambda x: x.min())
train['gp_brand_level3_min']=train.groupby('brand')['level3'].transform(lambda x: x.min())
train['gp_brand_level4_min']=train.groupby('brand')['level4'].transform(lambda x: x.min())

#众数


#####################################################test

#计数
test['gp_author_level1_cnt']=test.groupby('author')['level1'].transform(lambda x: x.count())
test['gp_author_level2_cnt']=test.groupby('author')['level2'].transform(lambda x: x.count())
test['gp_author_level3_cnt']=test.groupby('author')['level3'].transform(lambda x: x.count())
test['gp_author_level4_cnt']=test.groupby('author')['level4'].transform(lambda x: x.count())

test['gp_mall_level1_cnt']=test.groupby('mall')['level1'].transform(lambda x: x.count())
test['gp_mall_level2_cnt']=test.groupby('mall')['level2'].transform(lambda x: x.count())
test['gp_mall_level3_cnt']=test.groupby('mall')['level3'].transform(lambda x: x.count())
test['gp_mall_level4_cnt']=test.groupby('mall')['level4'].transform(lambda x: x.count())

test['gp_brand_level1_cnt']=test.groupby('brand')['level1'].transform(lambda x: x.count())
test['gp_brand_level2_cnt']=test.groupby('brand')['level2'].transform(lambda x: x.count())
test['gp_brand_level3_cnt']=test.groupby('brand')['level3'].transform(lambda x: x.count())
test['gp_brand_level4_cnt']=test.groupby('brand')['level4'].transform(lambda x: x.count())

#平均
test['gp_author_level1_mean']=test.groupby('author')['level1'].transform(lambda x: x.mean())
test['gp_author_level2_mean']=test.groupby('author')['level2'].transform(lambda x: x.mean())
test['gp_author_level3_mean']=test.groupby('author')['level3'].transform(lambda x: x.mean())
test['gp_author_level4_mean']=test.groupby('author')['level4'].transform(lambda x: x.mean())

test['gp_mall_level1_mean']=test.groupby('mall')['level1'].transform(lambda x: x.mean())
test['gp_mall_level2_mean']=test.groupby('mall')['level2'].transform(lambda x: x.mean())
test['gp_mall_level3_mean']=test.groupby('mall')['level3'].transform(lambda x: x.mean())
test['gp_mall_level4_mean']=test.groupby('mall')['level4'].transform(lambda x: x.mean())

test['gp_brand_level1_mean']=test.groupby('brand')['level1'].transform(lambda x: x.mean())
test['gp_brand_level2_mean']=test.groupby('brand')['level2'].transform(lambda x: x.mean())
test['gp_brand_level3_mean']=test.groupby('brand')['level3'].transform(lambda x: x.mean())
test['gp_brand_level4_mean']=test.groupby('brand')['level4'].transform(lambda x: x.mean())

#中位数
test['gp_author_level1_median']=test.groupby('author')['level1'].transform(lambda x: x.median())
test['gp_author_level2_median']=test.groupby('author')['level2'].transform(lambda x: x.median())
test['gp_author_level3_median']=test.groupby('author')['level3'].transform(lambda x: x.median())
test['gp_author_level4_median']=test.groupby('author')['level4'].transform(lambda x: x.median())

test['gp_mall_level1_median']=test.groupby('mall')['level1'].transform(lambda x: x.median())
test['gp_mall_level2_median']=test.groupby('mall')['level2'].transform(lambda x: x.median())
test['gp_mall_level3_median']=test.groupby('mall')['level3'].transform(lambda x: x.median())
test['gp_mall_level4_median']=test.groupby('mall')['level4'].transform(lambda x: x.median())

test['gp_brand_level1_median']=test.groupby('brand')['level1'].transform(lambda x: x.median())
test['gp_brand_level2_median']=test.groupby('brand')['level2'].transform(lambda x: x.median())
test['gp_brand_level3_median']=test.groupby('brand')['level3'].transform(lambda x: x.median())
test['gp_brand_level4_median']=test.groupby('brand')['level4'].transform(lambda x: x.median())

#方差
test['gp_author_level1_var']=test.groupby('author')['level1'].transform(lambda x: x.var())
test['gp_author_level2_var']=test.groupby('author')['level2'].transform(lambda x: x.var())
test['gp_author_level3_var']=test.groupby('author')['level3'].transform(lambda x: x.var())
test['gp_author_level4_var']=test.groupby('author')['level4'].transform(lambda x: x.var())

test['gp_mall_level1_var']=test.groupby('mall')['level1'].transform(lambda x: x.var())
test['gp_mall_level2_var']=test.groupby('mall')['level2'].transform(lambda x: x.var())
test['gp_mall_level3_var']=test.groupby('mall')['level3'].transform(lambda x: x.var())
test['gp_mall_level4_var']=test.groupby('mall')['level4'].transform(lambda x: x.var())

test['gp_brand_level1_var']=test.groupby('brand')['level1'].transform(lambda x: x.var())
test['gp_brand_level2_var']=test.groupby('brand')['level2'].transform(lambda x: x.var())
test['gp_brand_level3_var']=test.groupby('brand')['level3'].transform(lambda x: x.var())
test['gp_brand_level4_var']=test.groupby('brand')['level4'].transform(lambda x: x.var())


#标准差
test['gp_author_level1_std']=train.groupby('author')['level1'].transform(lambda x: x.std())
test['gp_author_level2_std']=test.groupby('author')['level2'].transform(lambda x: x.std())
test['gp_author_level3_std']=test.groupby('author')['level3'].transform(lambda x: x.std())
test['gp_author_level4_std']=test.groupby('author')['level4'].transform(lambda x: x.std())

test['gp_mall_level1_std']=test.groupby('mall')['level1'].transform(lambda x: x.std())
test['gp_mall_level2_std']=test.groupby('mall')['level2'].transform(lambda x: x.std())
test['gp_mall_level3_std']=test.groupby('mall')['level3'].transform(lambda x: x.std())
test['gp_mall_level4_std']=test.groupby('mall')['level4'].transform(lambda x: x.std())

test['gp_brand_level1_std']=test.groupby('brand')['level1'].transform(lambda x: x.std())
test['gp_brand_level2_std']=test.groupby('brand')['level2'].transform(lambda x: x.std())
test['gp_brand_level3_std']=test.groupby('brand')['level3'].transform(lambda x: x.std())
test['gp_brand_level4_std']=test.groupby('brand')['level4'].transform(lambda x: x.std())

#最大值
test['gp_author_level1_max']=test.groupby('author')['level1'].transform(lambda x: x.max())
test['gp_author_level2_max']=test.groupby('author')['level2'].transform(lambda x: x.max())
test['gp_author_level3_max']=test.groupby('author')['level3'].transform(lambda x: x.max())
test['gp_author_level4_max']=test.groupby('author')['level4'].transform(lambda x: x.max())

test['gp_mall_level1_max']=test.groupby('mall')['level1'].transform(lambda x: x.max())
test['gp_mall_level2_max']=test.groupby('mall')['level2'].transform(lambda x: x.max())
test['gp_mall_level3_max']=test.groupby('mall')['level3'].transform(lambda x: x.max())
test['gp_mall_level4_max']=test.groupby('mall')['level4'].transform(lambda x: x.max())

test['gp_brand_level1_max']=test.groupby('brand')['level1'].transform(lambda x: x.max())
test['gp_brand_level2_max']=test.groupby('brand')['level2'].transform(lambda x: x.max())
test['gp_brand_level3_max']=test.groupby('brand')['level3'].transform(lambda x: x.max())
test['gp_brand_level4_max']=test.groupby('brand')['level4'].transform(lambda x: x.max())

#最小值
test['gp_author_level1_min']=test.groupby('author')['level1'].transform(lambda x: x.min())
test['gp_author_level2_min']=test.groupby('author')['level2'].transform(lambda x: x.min())
test['gp_author_level3_min']=test.groupby('author')['level3'].transform(lambda x: x.min())
test['gp_author_level4_min']=test.groupby('author')['level4'].transform(lambda x: x.min())

test['gp_mall_level1_min']=test.groupby('mall')['level1'].transform(lambda x: x.min())
test['gp_mall_level2_min']=test.groupby('mall')['level2'].transform(lambda x: x.min())
test['gp_mall_level3_min']=test.groupby('mall')['level3'].transform(lambda x: x.min())
test['gp_mall_level4_min']=test.groupby('mall')['level4'].transform(lambda x: x.min())

test['gp_brand_level1_min']=test.groupby('brand')['level1'].transform(lambda x: x.min())
test['gp_brand_level2_min']=test.groupby('brand')['level2'].transform(lambda x: x.min())
test['gp_brand_level3_min']=test.groupby('brand')['level3'].transform(lambda x: x.min())
test['gp_brand_level4_min']=test.groupby('brand')['level4'].transform(lambda x: x.min())

#众数

In [7]:
#聚合特征
#orders_1h,orders_2h
###############################################train

#计数
train['gp_author_orders_1h_cnt']=train.groupby('author')['orders_1h'].transform(lambda x: x.count())
train['gp_author_orders_2h_cnt']=train.groupby('author')['orders_2h'].transform(lambda x: x.count())


train['gp_mall_orders_1h_cnt']=train.groupby('mall')['orders_1h'].transform(lambda x: x.count())
train['gp_mall_orders_2h_cnt']=train.groupby('mall')['orders_2h'].transform(lambda x: x.count())


train['gp_brand_orders_1h_cnt']=train.groupby('brand')['orders_1h'].transform(lambda x: x.count())
train['gp_brand_orders_2h_cnt']=train.groupby('brand')['orders_2h'].transform(lambda x: x.count())


#平均
train['gp_author_orders_1h_mean']=train.groupby('author')['orders_1h'].transform(lambda x: x.mean())
train['gp_author_orders_2h_mean']=train.groupby('author')['orders_2h'].transform(lambda x: x.mean())


train['gp_mall_orders_1h_mean']=train.groupby('mall')['orders_1h'].transform(lambda x: x.mean())
train['gp_mall_orders_2h_mean']=train.groupby('mall')['orders_2h'].transform(lambda x: x.mean())


train['gp_brand_orders_1h_mean']=train.groupby('brand')['orders_1h'].transform(lambda x: x.mean())
train['gp_brand_orders_2h_mean']=train.groupby('brand')['orders_2h'].transform(lambda x: x.mean())


#中位数
train['gp_author_orders_1h_median']=train.groupby('author')['orders_1h'].transform(lambda x: x.median())
train['gp_author_orders_2h_median']=train.groupby('author')['orders_2h'].transform(lambda x: x.median())


train['gp_mall_orders_1h_median']=train.groupby('mall')['orders_1h'].transform(lambda x: x.median())
train['gp_mall_orders_2h_median']=train.groupby('mall')['orders_2h'].transform(lambda x: x.median())


train['gp_brand_orders_1h_median']=train.groupby('brand')['orders_1h'].transform(lambda x: x.median())
train['gp_brand_orders_2h_median']=train.groupby('brand')['orders_2h'].transform(lambda x: x.median())


#方差
train['gp_author_orders_1h_var']=train.groupby('author')['orders_1h'].transform(lambda x: x.var())
train['gp_author_orders_2h_var']=train.groupby('author')['orders_2h'].transform(lambda x: x.var())


train['gp_mall_orders_1h_var']=train.groupby('mall')['orders_1h'].transform(lambda x: x.var())
train['gp_mall_orders_2h_var']=train.groupby('mall')['orders_2h'].transform(lambda x: x.var())


train['gp_brand_orders_1h_var']=train.groupby('brand')['orders_1h'].transform(lambda x: x.var())
train['gp_brand_orders_2h_var']=train.groupby('brand')['orders_2h'].transform(lambda x: x.var())



#标准差
train['gp_author_orders_1h_std']=train.groupby('author')['orders_1h'].transform(lambda x: x.std())
train['gp_author_orders_2h_std']=train.groupby('author')['orders_2h'].transform(lambda x: x.std())


train['gp_mall_orders_1h_std']=train.groupby('mall')['orders_1h'].transform(lambda x: x.std())
train['gp_mall_orders_2h_std']=train.groupby('mall')['orders_2h'].transform(lambda x: x.std())


train['gp_brand_orders_1h_std']=train.groupby('brand')['orders_1h'].transform(lambda x: x.std())
train['gp_brand_orders_2h_std']=train.groupby('brand')['orders_2h'].transform(lambda x: x.std())


#最大值
train['gp_author_orders_1h_max']=train.groupby('author')['orders_1h'].transform(lambda x: x.max())
train['gp_author_orders_2h_max']=train.groupby('author')['orders_2h'].transform(lambda x: x.max())


train['gp_mall_orders_1h_max']=train.groupby('mall')['orders_1h'].transform(lambda x: x.max())
train['gp_mall_orders_2h_max']=train.groupby('mall')['orders_2h'].transform(lambda x: x.max())


train['gp_brand_orders_1h_max']=train.groupby('brand')['orders_1h'].transform(lambda x: x.max())
train['gp_brand_orders_2h_max']=train.groupby('brand')['orders_2h'].transform(lambda x: x.max())


#最小值
train['gp_author_orders_1h_min']=train.groupby('author')['orders_1h'].transform(lambda x: x.min())
train['gp_author_orders_2h_min']=train.groupby('author')['orders_2h'].transform(lambda x: x.min())


train['gp_mall_orders_1h_min']=train.groupby('mall')['orders_1h'].transform(lambda x: x.min())
train['gp_mall_orders_2h_min']=train.groupby('mall')['orders_2h'].transform(lambda x: x.min())


train['gp_brand_orders_1h_min']=train.groupby('brand')['orders_1h'].transform(lambda x: x.min())
train['gp_brand_orders_2h_min']=train.groupby('brand')['orders_2h'].transform(lambda x: x.min())


#众数


#####################################################test

#计数
test['gp_author_orders_1h_cnt']=test.groupby('author')['orders_1h'].transform(lambda x: x.count())
test['gp_author_orders_2h_cnt']=test.groupby('author')['orders_2h'].transform(lambda x: x.count())


test['gp_mall_orders_1h_cnt']=test.groupby('mall')['orders_1h'].transform(lambda x: x.count())
test['gp_mall_orders_2h_cnt']=test.groupby('mall')['orders_2h'].transform(lambda x: x.count())


test['gp_brand_orders_1h_cnt']=test.groupby('brand')['orders_1h'].transform(lambda x: x.count())
test['gp_brand_orders_2h_cnt']=test.groupby('brand')['orders_2h'].transform(lambda x: x.count())


#平均
test['gp_author_orders_1h_mean']=test.groupby('author')['orders_1h'].transform(lambda x: x.mean())
test['gp_author_orders_2h_mean']=test.groupby('author')['orders_2h'].transform(lambda x: x.mean())


test['gp_mall_orders_1h_mean']=test.groupby('mall')['orders_1h'].transform(lambda x: x.mean())
test['gp_mall_orders_2h_mean']=test.groupby('mall')['orders_2h'].transform(lambda x: x.mean())


test['gp_brand_orders_1h_mean']=test.groupby('brand')['orders_1h'].transform(lambda x: x.mean())
test['gp_brand_orders_2h_mean']=test.groupby('brand')['orders_2h'].transform(lambda x: x.mean())


#中位数
test['gp_author_orders_1h_median']=test.groupby('author')['orders_1h'].transform(lambda x: x.median())
test['gp_author_orders_2h_median']=test.groupby('author')['orders_2h'].transform(lambda x: x.median())


test['gp_mall_orders_1h_median']=test.groupby('mall')['orders_1h'].transform(lambda x: x.median())
test['gp_mall_orders_2h_median']=test.groupby('mall')['orders_2h'].transform(lambda x: x.median())


test['gp_brand_orders_1h_median']=test.groupby('brand')['orders_1h'].transform(lambda x: x.median())
test['gp_brand_orders_2h_median']=test.groupby('brand')['orders_2h'].transform(lambda x: x.median())


#方差
test['gp_author_orders_1h_var']=test.groupby('author')['orders_1h'].transform(lambda x: x.var())
test['gp_author_orders_2h_var']=test.groupby('author')['orders_2h'].transform(lambda x: x.var())


test['gp_mall_orders_1h_var']=test.groupby('mall')['orders_1h'].transform(lambda x: x.var())
test['gp_mall_orders_2h_var']=test.groupby('mall')['orders_2h'].transform(lambda x: x.var())


test['gp_brand_orders_1h_var']=test.groupby('brand')['orders_1h'].transform(lambda x: x.var())
test['gp_brand_orders_2h_var']=test.groupby('brand')['orders_2h'].transform(lambda x: x.var())



#标准差
test['gp_author_orders_1h_std']=test.groupby('author')['orders_1h'].transform(lambda x: x.std())
test['gp_author_orders_2h_std']=test.groupby('author')['orders_2h'].transform(lambda x: x.std())


test['gp_mall_orders_1h_std']=test.groupby('mall')['orders_1h'].transform(lambda x: x.std())
test['gp_mall_orders_2h_std']=test.groupby('mall')['orders_2h'].transform(lambda x: x.std())


test['gp_brand_orders_1h_std']=test.groupby('brand')['orders_1h'].transform(lambda x: x.std())
test['gp_brand_orders_2h_std']=test.groupby('brand')['orders_2h'].transform(lambda x: x.std())


#最大值
test['gp_author_orders_1h_max']=test.groupby('author')['orders_1h'].transform(lambda x: x.max())
test['gp_author_orders_2h_max']=test.groupby('author')['orders_2h'].transform(lambda x: x.max())


test['gp_mall_orders_1h_max']=test.groupby('mall')['orders_1h'].transform(lambda x: x.max())
test['gp_mall_orders_2h_max']=test.groupby('mall')['orders_2h'].transform(lambda x: x.max())


test['gp_brand_orders_1h_max']=test.groupby('brand')['orders_1h'].transform(lambda x: x.max())
test['gp_brand_orders_2h_max']=test.groupby('brand')['orders_2h'].transform(lambda x: x.max())


#最小值
test['gp_author_orders_1h_min']=test.groupby('author')['orders_1h'].transform(lambda x: x.min())
test['gp_author_orders_2h_min']=test.groupby('author')['orders_2h'].transform(lambda x: x.min())


test['gp_mall_orders_1h_min']=test.groupby('mall')['orders_1h'].transform(lambda x: x.min())
test['gp_mall_orders_2h_min']=test.groupby('mall')['orders_2h'].transform(lambda x: x.min())


test['gp_brand_orders_1h_min']=test.groupby('brand')['orders_1h'].transform(lambda x: x.min())
test['gp_brand_orders_2h_min']=test.groupby('brand')['orders_2h'].transform(lambda x: x.min())

#众数

In [8]:
#合成特征

#####################train
train['baike_iou']=len(set(train['baike_id_1h'] & train['baike_id_2h']))/len(set(train['baike_id_1h'] | train['baike_id_2h']))
train['price_this']=train['price']+train['price_diff']
train['price_ratio']=train['price_diff']/train['price']
train['favorite_diff']=train['favorite_2h']-train['favorite_1h']
train['zhi_diff']=train['zhi_2h']-train['zhi_1h']
train['buzhi_diff']=train['buzhi_2h']-train['buzhi_1h']
train['orders_diff']=train['orders_2h']-train['orders_1h']
train['comments_diff']=train['comments_2h']-train['comments_1h']


#####################test
test['baike_iou']=len(set(test['baike_id_1h'] & test['baike_id_2h']))/len(set(test['baike_id_1h'] | test['baike_id_2h']))
test['price_this']=test['price']+test['price_diff']
test['price_ratio']=test['price_diff']/test['price']
test['favorite_diff']=test['favorite_2h']-test['favorite_1h']
test['zhi_diff']=test['zhi_2h']-test['zhi_1h']
test['buzhi_diff']=test['buzhi_2h']-test['buzhi_1h']
test['orders_diff']=test['orders_2h']-test['orders_1h']
test['comments_diff']=test['comments_2h']-test['comments_1h']


In [9]:
train['price_last']=train['price']-train['price_diff']
train['price_increase_ratio']=(train['price_last']+train['price_diff'])/train['price_last']
train['favorite_increase_ratio']=train['favorite_2h']/(1+train['favorite_1h'])
train['zhi_increase_ratio']=train['zhi_2h']/(1+train['zhi_1h'])
train['buzhi_increase_ratio']=train['buzhi_2h']/(1+train['buzhi_1h'])
train['orders_increase_ratio']=train['orders_2h']/(1+train['orders_1h'])
train['comments_increase_ratio']=train['comments_2h']/(1+train['comments_1h'])



test['price_last']=test['price']-test['price_diff']
test['price_increase_ratio']=(test['price_last']+test['price_diff'])/test['price_last']
test['favorite_increase_ratio']=test['favorite_2h']/(1+test['favorite_1h'])
test['zhi_increase_ratio']=test['zhi_2h']/(1+test['zhi_1h'])
test['buzhi_increase_ratio']=test['buzhi_2h']/(1+test['buzhi_1h'])
test['orders_increase_ratio']=test['orders_2h']/(1+test['orders_1h'])
test['comments_increase_ratio']=test['comments_2h']/(1+test['comments_1h'])

In [10]:
train

,article_id,date,baike_id_1h,price,price_diff,author,level1,level2,level3,level4,...,buzhi_diff,orders_diff,comments_diff,price_last,price_increase_ratio,favorite_increase_ratio,zhi_increase_ratio,buzhi_increase_ratio,orders_increase_ratio,comments_increase_ratio
0,356,1,104,5969.0,-597.8,21562,8,53,63,212,...,0,0,0,6566.8,0.908966,0.000000,0.0,0.00,0.000000,0.0000
1,388,1,219783,190.0,NaN,51522,14,15,580,0,...,-2,0,-4,NaN,NaN,0.615385,0.3,0.25,0.833333,0.6875
2,872,1,92240,139.0,20.0,69683,7,22,0,0,...,0,0,0,119.0,1.168067,0.000000,0.0,0.00,0.000000,0.0000
3,938,1,88894,99.0,NaN,63556,19,2,753,0,...,0,0,0,NaN,NaN,0.000000,0.0,0.00,0.000000,0.0000
4,1957,1,85380,779.0,10.0,63860,5,40,429,0,...,0,0,0,769.0,1.013004,0.000000,0.0,0.00,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807652,1804751,116,219783,99.0,0.0,44655,0,6,713,0,...,0,0,0,99.0,1.000000,0.000000,0.0,0.00,0.000000,0.0000
1807653,1805971,116,219783,217.0,NaN,68203,0,6,707,0,...,0,0,0,NaN,NaN,0.000000,0.0,0.00,0.000000,0.0000
1807654,1807148,116,219783,138.0,NaN,20391,19,4,763,0,...,0,0,0,NaN,NaN,0.000000,0.0,0.00,0.000000,0.0000
1807655,1807424,116,219783,559.0,9.0,22941,7,50,237,0,...,0,0,0,550.0,1.016364,0.000000,0.0,0.00,0.000000,0.0000


In [11]:
test

,article_id,date,baike_id_1h,price,price_diff,author,level1,level2,level3,level4,...,buzhi_diff,orders_diff,comments_diff,price_last,price_increase_ratio,favorite_increase_ratio,zhi_increase_ratio,buzhi_increase_ratio,orders_increase_ratio,comments_increase_ratio
0,1809360,117,219783,9.40,NaN,39424,19,31,744,0,...,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
1,1809857,117,219783,149.60,-38.77,51888,19,30,750,0,...,0,0,0,188.37,0.794182,0.0,0.0,0.0,0.0,0.0
2,1810275,117,219783,97.00,41.00,8649,13,13,664,0,...,0,0,0,56.00,1.732143,0.0,0.0,0.0,0.0,0.0
3,1810622,117,68785,5.42,2.50,52372,3,5,27,798,...,0,0,0,2.92,1.856164,0.5,0.0,0.0,0.5,0.0
4,1811139,117,219783,79.00,NaN,20410,13,96,146,0,...,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149584,1955932,123,219783,123.50,18.05,69329,8,57,84,0,...,0,0,0,105.45,1.171171,0.0,0.0,0.0,0.0,0.0
149585,1956580,123,29553,2.40,1.45,71272,19,2,754,1199,...,0,0,0,0.95,2.526316,0.0,0.0,0.0,0.0,0.0
149586,1956675,123,219783,310.80,NaN,665,13,96,1,0,...,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
149587,1956922,123,49355,99.00,10.00,55420,5,43,319,0,...,0,0,0,89.00,1.112360,0.0,0.0,0.0,0.0,0.0


In [12]:
x_train = train.drop(['article_id','orders_3h_15h'],axis=1)
x_test = test.drop(['article_id'],axis=1)
y_train = train['orders_3h_15h']

In [13]:
y_train.unique()

array([0, 5, 4, 3, 1, 6, 2, 8, 7, 9], dtype=int64)

In [27]:
def cv_model(clf, train_x, train_y, test_x, clf_name='lgb'):
    folds = 5
    seed = 2021
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        train_matrix = clf.Dataset(trn_x, label=trn_y)
        valid_matrix = clf.Dataset(val_x, label=val_y)

        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'mse',
            'min_child_weight': 5,
            'max_depth': 10,
            'num_leaves': 2 ** 4,
            'lambda_l2': 10,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.9,
            'bagging_freq': 4,
            'learning_rate': 0.1,
            'seed': 2021,
            'nthread': 28,
            'n_jobs':-1,
            'silent': True,
            'verbose': -1,
        }

        model = clf.train(params, train_matrix, 5000, valid_sets=[train_matrix, valid_matrix], verbose_eval=500,early_stopping_rounds=200)
        val_pred = model.predict(val_x, num_iteration=model.best_iteration)
        test_pred = model.predict(test_x, num_iteration=model.best_iteration)

        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(mean_squared_error(val_y, val_pred))
        
        print(cv_scores)
        
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

In [28]:
lgb_train, lgb_test = cv_model(lgb, x_train, y_train, x_test)

************************************ 1 ************************************
Training until validation scores don't improve for 200 rounds
[500]	training's l2: 1.22301	valid_1's l2: 1.23671
[1000]	training's l2: 1.192	valid_1's l2: 1.22362
[1500]	training's l2: 1.16783	valid_1's l2: 1.21607
[2000]	training's l2: 1.14716	valid_1's l2: 1.21082
[2500]	training's l2: 1.12863	valid_1's l2: 1.207
[3000]	training's l2: 1.11161	valid_1's l2: 1.20412
[3500]	training's l2: 1.0957	valid_1's l2: 1.20193
[4000]	training's l2: 1.08131	valid_1's l2: 1.20041
[4500]	training's l2: 1.06732	valid_1's l2: 1.19874
[5000]	training's l2: 1.05395	valid_1's l2: 1.19734
Did not meet early stopping. Best iteration is:
[5000]	training's l2: 1.05395	valid_1's l2: 1.19734
[1.1973364466150944]
************************************ 2 ************************************
[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=-1. Current value: num_threads=-1
[LightGBM] [Warning] Unknown pa

In [29]:
lgb_test=lgb_test.clip(0)

In [30]:
sample_submit=pd.DataFrame()

In [31]:
## 预测结果
sample_submit['article_id']=test['article_id']
sample_submit['orders_3h_15h'] = lgb_test
#sample_submit['loan_default'] = sample_submit['loan_default'].apply(lambda x:1 if x>0.25 else 0).values
sample_submit.to_csv('eee.csv', index=False)